# Classification de textes avec les Transformers

<p align="center">
  <a href="https://raw.githubusercontent.com/auduvignac/llm-finetuning/refs/heads/main/notebooks/project/finetuning-projet.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Ouvrir dans Google Colab"/>
  </a>
</p>

Ce projet porte sur une tâche de **classification de textes** appliquée au **jeu de données IMDB** (analyse de sentiments ou *sentimental analysis*).  
Ce travail s'appuiera sur les **architectures de type encodeur**, en particulier l’un des modèles les plus connus : **BERT** (et sa variante légère **DistilBERT**).

## Présentation

Le projet consiste à utiliser la librairie `datasets` pour le chargement des données et les `tokenizers` pour le prétraitement des textes.

La librairie **Transformers** propose une API simple pour utiliser des modèles pré-entraînés tels que **BERT** ou **GPT**. Elle facilite leur téléchargement, leur ré-entraînement et leur intégration, tout en réduisant les coûts de calcul et en restant compatible avec **PyTorch, TensorFlow et JAX**.

## Objectif du projet

Dans le cadre de ce projet, l’expérimentation portera sur la librairie **Hugging Face** afin de :
- Charger et adapter un modèle de type **BERT** à une tâche de classification de textes ;
- Évaluer ses performances sur le dataset **IMDB** ;
- Analyser les résultats et discuter des choix réalisés (modèle, preprocessing, paramètres, etc.).

Le travail sera guidé par les interrogations suivantes :

- Bien que tous ces modèles reposent sur l’architecture **Transformer**, quelles en sont les spécificités ?
- Quel format d’entrée est attendu par le modèle ?
- Quels types de sorties génère-t-il ?
- Le modèle peut-il être utilisé tel quel ou doit-il être adapté à la tâche considérée ?

Ces questions constituent une part essentielle du travail quotidien d’un chercheur en NLP et seront examinées dans le cadre de ce projet de *fine-tuning*.

In [1]:
!wget -q https://raw.githubusercontent.com/auduvignac/llm-finetuning/refs/heads/main/setup_env.py -O setup_env.py
%run setup_env.py

⚡ Exécution sur Colab : vérification des dépendances...
✅ datasets déjà présent.
✅ matplotlib déjà présent.
✅ numpy déjà présent.
✅ tabulate déjà présent.
✅ torch déjà présent.
✅ tqdm déjà présent.
✅ transformers déjà présent.


In [2]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']
import matplotlib.pyplot as plt

from transformers import DistilBertTokenizer
from tqdm.notebook import tqdm


import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
from torch.utils.data import DataLoader
from tabulate import tabulate
from datasets import load_dataset

from tqdm.notebook import tqdm

# If the machine you run this on has a GPU available with CUDA installed,
# use it. Using a GPU for learning often leads to huge speedups in training.
# See https://developer.nvidia.com/cuda-downloads for installing CUDA
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

## Chargement du jeu d'entraînement

La phase initiale consiste à procéder au chargement du jeu d'entraînement au moyen de l'instruction suivante :
```python
dataset = load_dataset("scikit-learn/imdb", split="train")
```

In [3]:
dataset = load_dataset("scikit-learn/imdb", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

IMDB Dataset.csv:   0%|          | 0.00/66.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Après le chargement, il convient de procéder à l'affichage du jeu de données afin d'en examiner la structure.

In [5]:
print(dataset)

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})


L'exécution de la commande `print(dataset)` renvoie la description suivante du jeu de données :

```
Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})
```

Le corpus est constitué de 50 000 instances, chacune décrite par deux attributs : le texte de la critique (*review*) et son étiquette de polarité (*sentiment*).


## Préparation des entrées du modèle

Le format d'entrée attendu par **BERT** peut être considéré comme « sur-spécifié », notamment lorsqu'il s'agit de tâches ciblées telles que la *sequence classification*, le *word tagging* ou la *paraphrase detection*. Ce format repose sur plusieurs contraintes :

* l'ajout de *special tokens* au début et à la fin de chaque phrase ;
* l'application d'un *padding* et d'une troncature afin de ramener toutes les phrases à une longueur constante ;
* la distinction explicite entre *real tokens* et *padding tokens* au moyen de l'*attention mask*.


<p align="center">
<img src="https://raw.githubusercontent.com/auduvignac/llm-finetuning/97e2b676168167ed5ff624f1ad98589c63919d5d/figures/bert_encoding_process.png" width="600">
</p>

La figure ci-dessus illustre le processus de préparation et de traitement des séquences textuelles dans le modèle **BERT**.

1. **Tokenisation et ajout de tokens spéciaux**
   La séquence textuelle est d'abord segmentée en *tokens*. Deux *special tokens* sont ajoutés : `[CLS]`, placé en début de séquence et utilisé comme représentation globale, ainsi que `[SEP]`, placé en fin de séquence et jouant un rôle de séparateur.

2. **Normalisation de la longueur des séquences**
   Afin d'assurer une longueur uniforme entre les exemples, les séquences sont complétées par des *\[PAD] tokens* ou, le cas échéant, tronquées à une taille maximale prédéfinie (*MAX\_LEN*).

3. **Attention mask**
   Un vecteur binaire, appelé *attention mask*, est associé à chaque séquence. La valeur `1` indique un *real token* tandis que la valeur `0` correspond à un *padding token*. Ce mécanisme permet au modèle d'ignorer les positions de remplissage au cours de l'entraînement et de l'inférence.

4. **Propagation à travers les couches du Transformer**
   Les représentations vectorielles des tokens traversent successivement les différentes couches de l'encodeur Transformer (ici, 12 couches). Chaque couche applique un mécanisme d'auto-*attention*, permettant de capturer les dépendances contextuelles entre tokens.

5. **Production de la prédiction**
   À l'issue de la dernière couche, seule la représentation associée au token `[CLS]` est retenue. Celle-ci est transmise au classificateur, qui produit la prédiction finale (par exemple, la polarité d'une critique dans une tâche de *sentiment analysis*).

**Remarque :** Pour des raisons liées aux coûts de calcul, le modèle [DistilBERT](https://huggingface.co/docs/transformers/model_doc/distilbert) sera utilisé. Ce dernier constitue une version réduite d'environ 40 % par rapport à BERT, tout en conservant près de 95 % des performances du modèle original.

Afin de préparer les données textuelles pour le modèle, il est nécessaire d'instancier un *tokenizer*. Celui-ci a pour rôle de segmenter les phrases en unités élémentaires (*tokens*) et de les convertir en identifiants numériques exploitables par le modèle. Dans le cadre de ce projet, il sera fait usage du `DistilBertTokenizer` pré-entraîné, correspondant au modèle *distilbert-base-uncased*.

```python
tokenizer = DistilBertTokenizer.from_pretrained(
    "distilbert-base-uncased", do_lower_case=True
)
```


In [6]:
tokenizer = DistilBertTokenizer.from_pretrained(
    "distilbert-base-uncased", do_lower_case=True
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

L'étape suivante consiste à examiner la manière dont le *tokenizer* traite la séquence.

1. Définition d'un message exemple

In [7]:
message = "hello my name is kevin"

2. *Tokenization* de la séquence

In [9]:
tok = tokenizer.tokenize(message)
print("Tokens dans la séquence:", tok)

Tokens dans la séquence: ['hello', 'my', 'name', 'is', 'kevin']


3. Encodage en identifiants numériques

In [10]:
enc = tokenizer.encode(tok)

4. Mise en correspondance entre token IDs et tokens

In [11]:
table = np.array([
    enc,
    [tokenizer.ids_to_tokens[w] for w in enc],
]).T

5. Affichage des résultats encodés

In [12]:
print("Données d'entrée encodées:")
print(tabulate(table, headers=["Token IDs", "Tokens"], tablefmt="fancy_grid"))

Données d'entrée encodées:
╒═════════════╤══════════╕
│   Token IDs │ Tokens   │
╞═════════════╪══════════╡
│         101 │ [CLS]    │
├─────────────┼──────────┤
│        7592 │ hello    │
├─────────────┼──────────┤
│        2026 │ my       │
├─────────────┼──────────┤
│        2171 │ name     │
├─────────────┼──────────┤
│        2003 │ is       │
├─────────────┼──────────┤
│        4901 │ kevin    │
├─────────────┼──────────┤
│         102 │ [SEP]    │
╘═════════════╧══════════╛


Le tableau obtenu met en évidence la correspondance entre les *tokens* et leurs identifiants numériques.
Les *tokens* spéciaux `[CLS]` et `[SEP]` encadrent la séquence, tandis que chaque mot du texte est associé à un identifiant unique destiné au traitement par le modèle.

Les special tokens `[CLS]` et `[SEP]` sont ajoutés automatiquement par HuggingFace afin de structurer la séquence d'entrée pour le modèle BERT et ses variantes.

Le token `[CLS]`, inséré en début de séquence, constitue une représentation globale de la phrase. Sa sortie est utilisée par la couche de classification pour produire la prédiction finale (par exemple en *sentiment analysis*).

Le token `[SEP]`, placé en fin de séquence (ou comme séparateur entre deux phrases), sert à marquer les frontières de segments textuels. Il est essentiel dans les tâches nécessitant plusieurs entrées, comme la comparaison de paires de phrases (*natural language inference*, *paraphrase detection*).

Ainsi, ces *tokens* spéciaux garantissent une structuration normalisée des entrées, indispensable au fonctionnement du modèle.